In [ ]:
#This contains code to automate the Funding calculation process. The Table and Column names are generic for security reasons.
#Replace with your Database tables and columns.

import os
import pandas as pd
import pyodbc
from datetime import datetime

#Define the connection string
connection_string = 'Your_Connection_String'

#Create a connection to the databae
def get_connection():
    conn = pyodbc.connect(connection_string)
    return conn

#Fetch data for a specific user and course instance
def fetch_data(user_id, course_instance_code, conn):
    query = f"""
    SELECT
        u.User_ID,
        u.Prior_Education_Level,
        u.Birth_Date,
        e.Work_Status,
        c.Course_Instance_Code,
        c.Funding_Status
    FROM
        Users u
    JOIN
        Employment_Status e ON u.User_Code = e.User_Code
    JOIN
        User_Courses c ON u.User_Code = c.User_Code
    WHERE
        u.User_Code = '{user_id}' AND
        c.Course_Instance_Code = '{course_instance_code}'
    """
    return pd.read_sql(query, conn)

#Fetch the course level for a given course instance
def fetch_course_level(course_instance_code, conn):
    query = f"""
    SELECT
        Course_Level
    FROM
        Course_Details
    WHERE
        Course_Instance_Code = '{course_instance_code}'
    """
    result = pd.read_sql(query, conn)
    if not result.empty:
        return result['Course_Level'].iloc[0]
    return None

#Calculate age as of 31st August 2024
def calculate_age(date_of_birth):
    reference_date = datetime(2024, 8, 1)
    if isinstance(date_of_birth, pd.Timestamp):
        birth_date = date_of_birth.to_pydatetime()
    else:
        birth_date = datetime.strptime(date_of_birth, "%Y-%m-%d")
    age = reference_date.year - birth_date.year - ((reference_date.month, reference_date.day) < (birth_date.month, birth_date.day))
    return age

#Classify employment status
def classify_employment_status(work_status):
    if work_status in ['11', '12']:
        return 'Unemployed'
    elif work_status == '10':
        return 'Employed'
    else:
        return 'Unknown'

#Determine funding based on age category, prior education level, employment status, and course level
def determine_funding():
    #Step 1: Input data
    user_id = input("Enter user ID: ").strip()
    course_instance_code = input("Enter course instance code: ").strip()

    #Create a connection to the database
    conn = get_connection()

    #Fetch data for the given user and course instance
    data = fetch_data(user_id, course_instance_code, conn)
    print("\nFetched Data:")
    print(data)

    if data.empty:
        return "No data available for the given user ID and course instance code."

    #Calculate age and determine age category
    data['age'] = data['Birth_Date'].apply(calculate_age)
    data['age_category'] = data['age'].apply(lambda x: '16-18' if x < 19 else '19-23' if x < 24 else '24+')

    #Print age and age category
    print("\nCalculated Age and Age Category:")
    print(data[['age', 'age_category']])

    #Classify employment status
    data['work_status_classified'] = data['Work_Status'].apply(classify_employment_status)
    print("\nClassified Work Status:")
    print(data[['Work_Status', 'work_status_classified']])

    #Determine prior education level
    prior_education = data['Prior_Education_Level'].iloc[0]
    print(f"\nPrior Education Level: {prior_education}")

    #Fetch course level
    course_level = fetch_course_level(course_instance_code, conn)
    print(f"\nCourse Level: {course_level}")

    if course_level is None:
        return "Course level not found."

    #Step 2: Determine funding based on age category
    age_category = data['age_category'].iloc[0]

    if age_category == '16-18':
        print("\nApplying 16-18 waiver")
        return "16-18 waiver"

    if age_category == '19-23':
        if prior_education in ['9', '99', '7', '1']:
            employment_status = data['work_status_classified'].iloc[0]
            if employment_status == 'Unemployed':
                if course_level == '3':
                    return "Funding_Level_3_Unemployed"
                elif course_level == '4':
                    return "Loan_Pay_No_Funding"
            elif employment_status == 'Employed':
                annual_wage = float(input("Enter annual wage (numeric value): ").strip())
                print(f"\nAnnual Wage: {annual_wage}")
                if annual_wage < 32200:
                    if course_level == '3':
                        return "Funding_Level_3_Employed_Below_32k"
                    elif course_level == '4':
                        return "Loan_Pay_No_Funding"
                elif annual_wage >= 32200:
                    return "Loan_Pay_No_Funding"

        if prior_education == '2':
            employment_status = data['work_status_classified'].iloc[0]
            if employment_status == 'Unemployed':
                if course_level == '3':
                    return "Funding_Level_3_Unemployed"
                elif course_level == '4':
                    return "Loan_Pay_No_Funding"
            elif employment_status == 'Employed':
                annual_wage = float(input("Enter annual wage (numeric value): ").strip())
                print(f"\nAnnual Wage: {annual_wage}")
                if annual_wage < 32200:
                    if course_level == '3':
                        return "Funding_Level_3_Employed_Below_32k"
                    elif course_level == '4':
                        return "Loan_Pay_No_Funding"
                elif annual_wage >= 32200:
                    return "Loan_Pay_No_Funding"

        if prior_education in ['3', '10', '11', '12', '13']:
            return "Loan_Pay_No_Funding"

    if age_category == '24+':
        if prior_education in ['9', '99', '7', '1']:
            employment_status = data['work_status_classified'].iloc[0]
            if employment_status == 'Unemployed':
                if course_level == '3':
                    return "Loan_Pay_No_Funding"
                elif course_level == '4':
                    return "Loan_Pay_No_Funding"
            elif employment_status == 'Employed':
                annual_wage = float(input("Enter annual wage (numeric value): ").strip())
                print(f"\nAnnual Wage: {annual_wage}")
                if annual_wage < 32200:
                    if course_level == '3':
                        return "Funding_Level_3_Employed_Below_32k"
                    elif course_level == '4':
                        return "Loan_Pay_No_Funding"
                elif annual_wage >= 32200:
                    return "Loan_Pay_No_Funding"

    return "Not Funded"

#Example:
funding = determine_funding()
print(f"\nThe funding type is: {funding}")